## Code to construct basic power grid networks and solve using DC OPF, with specific focus on analysis of Locational Marginal Prices (LMP)
### @author: kyribaker
### From the paper https://arxiv.org/pdf/2403.19032

In [73]:
import cvxpy as cp
import numpy as np

# DEFINE NETWORK PARAMETERS ````````
nbus = 9 # number of buses
ngen = 4 # number of generators
nline =10 # number of lines
loads = [0, 5000, 1000, 0, 600, 0, 300, 0, 200] # by default, a load is at each bus (set to zero if no load)
genbus = [0, 3, 5, 7] # bus locations of generators
gencost = [20, 20, 20, 20] # cost of each generator
genlim_down = [0]*ngen # lower generation limits
genlim_up = [2200, 6000, 2200, 1000] # upper generation limits

# adjacency matrix - 0 if nodes i and j are not connected, 1 if they are.
Ao = np.zeros((nbus,nbus))

# Adapted from 6 node example from Pritchard: A Single-Settlement, Energy-Only Electric Power Market
#for Unpredictable and Intermittent Participants
Ao[0,1] = 1
Ao[0,2] = 1
Ao[1,3] = 1
Ao[2,3] = 1 
Ao[3,4] = 1 
Ao[4,5] = 1 
Ao[5,6] = 1
Ao[6,7] = 1
Ao[0,7] = 1
Ao[7,8] = 1
# mesh 
#Ao[1,3] = 1
#Ao[0,5] = 1


A = np.maximum(Ao, Ao.transpose()) # ensure A is symmetric

assert sum(sum(A)) == nline*2, "Did you form the Adjacency matrix correctly? Or use the wrong number of lines?"
assert sum(np.diag(A)) == 0, "Did you form the Adjacency matrix correctly?"

# DEFINE CVXPY VARIABLES ````````
Pg = cp.Variable(ngen)
t = cp.Variable(nbus)

# line susceptance matrix (just assume equal reactances for now)
B = A

# line limits : 12345 indicates no flow limit
# Limits between buses 1 and 6 (indices 0 and 5), and 2 and 4 (indices 1 and 3)
L = 12345*np.ones((nbus, nbus))
L[1,3] = 2000; L[3,1] = 2000;
#L[0,2] = 100; L[2,0] = 100;
#L[5,6] = 20; L[6,5] = 20

In [74]:
# DC OPF
obj = 0
for i in range(ngen):
    obj += gencost[i]*Pg[i]
    
objective = cp.Minimize(obj)
constraints = []
LMP_idx = [] # to store location of power balance constraints
flow_idx = [] # to store location of lineflow constraints

# Generator limits
constraints += [Pg >= genlim_down] # Minimum generation limit
constraints += [Pg <= genlim_up] # Maximum generation limit

# Power Balance and Line flow limits
for i in range(nbus):
    tmpcon = 0
    if i in genbus: 
        tmpcon = -Pg[genbus.index(i)] + loads[i]
    else: 
        tmpcon = loads[i]
    for j in range(nbus):
        tmpcon += B[i,j]*(t[i]-t[j]) # add up all power flows coming from bus i
        
        if L[i,j] != 12345: # Line limit exists on this line
            constraints += [B[i,j]*(t[i]-t[j]) <= L[i,j]]
            flow_idx += [len(constraints)]
            constraints += [B[j,i]*(t[j]-t[i]) <= L[i,j]]
            flow_idx += [len(constraints)]
            
    constraints += [tmpcon == 0] # power balance
    LMP_idx += [len(constraints)] # indices of power balance constraints

prob = cp.Problem(objective, constraints)

results = prob.solve()
print(prob.status)
print("Total Cost: $" + str(round(objective.value, 2)))

# Get LMPs from solver
busLMP = [constraints[i-1].dual_value for i in LMP_idx]

# If congestion is present, these multipliers should be > 0
lineLMP = [constraints[i-1].dual_value for i in flow_idx]

print('\n`````````DUAL VARIABLE INFO`````````')

for i in range(nbus):
    print('Price at bus ' + str(i) + ': $' + str(round(busLMP[i],3)))

if sum(lineLMP) > 0.2:
    print('Congestion present!')
else:
    print('No congestion present!')

print('\n`````````GENERATOR INFO`````````')

for i in range(nbus):
    if i in genbus: 
        print('Gen '+ str(i) + ': ' + str(round(Pg[genbus.index(i)].value)) + " MWh ($" + str(gencost[genbus.index(i)]) + "/MWh), gets paid: $", 
              str(round(Pg[genbus.index(i)].value*busLMP[i],2)))
        if abs(round(Pg[genbus.index(i)].value) - genlim_up[genbus.index(i)]) > .1 and abs(round(Pg[genbus.index(i)].value) - genlim_down[genbus.index(i)]) > 0.1:
            print('MARGINAL')

# Calculate and display power flows on each line
print('\n`````````LINE FLOW INFO`````````')
for i in range(nbus):
    for j in range(i + 1, nbus):  # Avoid double-counting lines
        if A[i, j] == 1:  # Line exists
            flow = B[i, j] * (t[i].value - t[j].value)
            print(f"Power flow on line {i}-{j}: {round(flow, 2)} MW")

infeasible


TypeError: type NoneType doesn't define __round__ method